In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

In [21]:
ace = pd.DataFrame()
for name in os.listdir('proc_data/total')[:-1]:
    ins_data = pd.read_csv(f'proc_data/total/{name}', index_col=0, parse_dates=['date'])
    ace = pd.concat([ace, ins_data], axis=0)

problem = pd.read_csv("proc_data/problem.csv", index_col=0, parse_dates=['date'])
problem.head()

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag
date,,,,,,,
01-01 00:00,1.225355,32956.035503,443.036509,5.142349,1.988692,-12.914000,14.143320
01-01 03:00,1.613686,55713.597041,431.723491,3.574822,-2.570586,-9.271053,11.855373
01-01 06:00,1.191851,80571.958333,432.390536,4.361542,-5.262113,-7.125196,10.517149
01-01 09:00,1.100000,149231.295858,428.213609,3.533574,-6.503805,4.220485,9.589148
01-01 12:00,1.100000,77718.396450,413.764024,6.511308,-6.137467,-0.664426,9.313183


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

print(tf.__version__, (tf.test.gpu_device_name() if tf.test.is_gpu_available() else None))

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

2.0.0-rc0 /device:GPU:0


In [22]:
ace = ace.reset_index().drop(np.where(np.round(ace.iloc[:, -2].values) == -10000)[0]).set_index('date')

In [23]:
ace_x, ace_y = ace.iloc[:, :-1].values, \
    keras.utils.to_categorical(ace.target.values, 10)
problem_x = tf.constant(problem.values[:, :, np.newaxis])

x_train, x_test, y_train, y_test = \
    train_test_split(ace_x, ace_y, test_size=0.2, random_state=1, shuffle=False)
x_train, x_val, y_train, y_val = \
    train_test_split(x_train, y_train, test_size=0.2, random_state=1)

ace_x, ace_y, x_train, y_train, x_val, y_val, x_test, y_test = \
    tf.constant(ace_x[:, :, np.newaxis]), tf.constant(ace_y), \
    tf.constant(x_train[:, :, np.newaxis]), tf.constant(y_train), \
    tf.constant(x_val[:, :, np.newaxis]), tf.constant(y_val), \
    tf.constant(x_test[:, :, np.newaxis]), tf.constant(y_test)

print(ace_x.shape, ace_y.shape, problem_x.shape)
print(x_train.shape, x_val.shape, x_test.shape)

(40903, 7, 1) (40903, 10) (2920, 7, 1)
(26177, 7, 1) (6545, 7, 1) (8181, 7, 1)


In [31]:
scaler = MinMaxScaler()

ace_x, ace_y = np.concatenate((np.array(scaler.fit_transform(ace.iloc[:, :3])), np.absolute(ace.iloc[:, 3:-1].values)), 1), \
    keras.utils.to_categorical(ace.target.values, 10)
problem_x = tf.constant(np.concatenate((np.array(scaler.fit_transform(problem.iloc[:, :3])), np.absolute(problem.iloc[:, 3:].values)), 1)[:, :, np.newaxis])

x_train, x_test, y_train, y_test = \
    train_test_split(ace_x, ace_y, test_size=0.2, random_state=1, shuffle=False)
x_train, x_val, y_train, y_val = \
    train_test_split(x_train, y_train, test_size=0.2, random_state=1)

ace_x, ace_y, x_train, y_train, x_val, y_val, x_test, y_test = \
    tf.constant(ace_x[:, :, np.newaxis]), tf.constant(ace_y), \
    tf.constant(x_train[:, :, np.newaxis]), tf.constant(y_train), \
    tf.constant(x_val[:, :, np.newaxis]), tf.constant(y_val), \
    tf.constant(x_test[:, :, np.newaxis]), tf.constant(y_test)

print(ace_x.shape, ace_y.shape, problem_x.shape)
print(x_train.shape, x_val.shape, x_test.shape)

(40903, 7, 1) (40903, 10) (2920, 7, 1)
(26177, 7, 1) (6545, 7, 1) (8181, 7, 1)


In [40]:
optim = keras.optimizers.Adam()
loss_fn = keras.losses.MeanSquaredError()

model = keras.models.Sequential()
model.add(layers.SimpleRNN(64, input_shape=(7, 1)))
model.add(layers.Dense(48, activation='tanh'))
model.add(layers.Dense(10, activation='linear'))
model.compile(optimizer=optim, loss=loss_fn, metrics=['acc'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_10 (SimpleRNN)    (None, 64)                4224      
_________________________________________________________________
dense_20 (Dense)             (None, 48)                3120      
_________________________________________________________________
dense_21 (Dense)             (None, 10)                490       
Total params: 7,834
Trainable params: 7,834
Non-trainable params: 0
_________________________________________________________________


In [28]:
early_stop = EarlyStopping(monitor='loss', patience=2)

hist = model.fit(x_train, y_train, batch_size=16, epochs=50, callbacks=[early_stop], validation_data=(x_val, y_val))
hist.params

Train on 26177 samples, validate on 6545 samples
Epoch 1/50
26177/26177 [==============================] - 17s 639us/sample - loss: 0.0760 - acc: 0.3448 - val_loss: 0.0720 - val_acc: 0.4069
Epoch 2/50
26177/26177 [==============================] - 16s 604us/sample - loss: 0.0708 - acc: 0.3933 - val_loss: 0.0705 - val_acc: 0.3976
Epoch 3/50
26177/26177 [==============================] - 16s 607us/sample - loss: 0.0698 - acc: 0.4047 - val_loss: 0.0721 - val_acc: 0.3969
Epoch 4/50
26177/26177 [==============================] - 17s 639us/sample - loss: 0.0695 - acc: 0.4128 - val_loss: 0.0720 - val_acc: 0.4018
Epoch 5/50
26177/26177 [==============================] - 16s 616us/sample - loss: 0.0694 - acc: 0.4144 - val_loss: 0.0690 - val_acc: 0.4188
Epoch 6/50
26177/26177 [==============================] - 17s 651us/sample - loss: 0.0692 - acc: 0.4136 - val_loss: 0.0688 - val_acc: 0.4174
Epoch 7/50
26177/26177 [==============================] - 17s 631us/sample - loss: 0.0691 - acc: 0.4194 -

{'batch_size': 16,
 'epochs': 50,
 'steps': 1637,
 'samples': 26177,
 'verbose': 0,
 'do_validation': True,
 'metrics': ['loss', 'acc', 'val_loss', 'val_acc']}

In [29]:
pred = model.predict_classes(problem_x)
loss, acc = model.evaluate(x_test, y_test, verbose=0)

timestamp = datetime.now().strftime('%Y-%m%d_%H%M')
pred = pd.DataFrame(pred.reshape(365, 8), index=range(1, 366), columns=[f"kp_{_}h" for _ in range(0, 24, 3)])
pred.to_csv(f"models/rnn{timestamp}(loss{round(float(loss), 3)}_acc{round(float(acc), 2)}).csv")

print(f"loss: {loss} - acc: {acc}")

loss: 0.06955213378124461 - acc: 0.38442733883857727


In [41]:
model.fit(ace_x, ace_y, batch_size=16, epochs=25, verbose=2)
loss, acc = model.evaluate(x_test, y_test, verbose=0)
timestamp = datetime.now().strftime('%Y-%m%d_%H%M')
pred = model.predict_classes(problem_x)
pred = pd.DataFrame(pred.reshape(365, 8), index=range(1, 366), columns=[f"kp_{_}h" for _ in range(0, 24, 3)])
pred.to_csv(f"models/ace_rnn{timestamp}(loss{round(float(loss), 3)}_acc{round(float(acc), 2)}).csv")

Train on 40903 samples
Epoch 1/25
40903/40903 - 22s - loss: 0.0755 - acc: 0.3490
Epoch 2/25
40903/40903 - 21s - loss: 0.0716 - acc: 0.3854
Epoch 3/25
40903/40903 - 22s - loss: 0.0709 - acc: 0.3893
Epoch 4/25
40903/40903 - 22s - loss: 0.0706 - acc: 0.3951
Epoch 5/25
40903/40903 - 22s - loss: 0.0704 - acc: 0.3944
Epoch 6/25
40903/40903 - 23s - loss: 0.0701 - acc: 0.3973
Epoch 7/25
40903/40903 - 22s - loss: 0.0701 - acc: 0.3982
Epoch 8/25
40903/40903 - 22s - loss: 0.0700 - acc: 0.4004
Epoch 9/25
40903/40903 - 22s - loss: 0.0698 - acc: 0.4002
Epoch 10/25
40903/40903 - 22s - loss: 0.0700 - acc: 0.3993
Epoch 11/25
40903/40903 - 22s - loss: 0.0698 - acc: 0.4007
Epoch 12/25
40903/40903 - 22s - loss: 0.0697 - acc: 0.4026
Epoch 13/25
40903/40903 - 22s - loss: 0.0697 - acc: 0.4017
Epoch 14/25
40903/40903 - 22s - loss: 0.0696 - acc: 0.4044
Epoch 15/25
40903/40903 - 22s - loss: 0.0697 - acc: 0.4039
Epoch 16/25
40903/40903 - 22s - loss: 0.0696 - acc: 0.4059
Epoch 17/25
40903/40903 - 22s - loss: 0.06